In [1]:
!pip install scapy


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 23.9 MB/s eta 0:00:00


Capture Packets:

In [9]:
from scapy.all import sniff, IP, TCP, UDP, ICMP
import pandas as pd
import time

# List to store captured packets
packet_list = []

def packet_handler(packet):
    if packet.haslayer(IP):  # Only process IP packets
        packet_data = {
            "timestamp": time.time(),
            "src_ip": packet[IP].src,
            "dst_ip": packet[IP].dst,
            "src_port": packet[TCP].sport if packet.haslayer(TCP) else (packet[UDP].sport if packet.haslayer(UDP) else None),
            "dst_port": packet[TCP].dport if packet.haslayer(TCP) else (packet[UDP].dport if packet.haslayer(UDP) else None),
            "protocol": "TCP" if packet.haslayer(TCP) else "UDP" if packet.haslayer(UDP) else "ICMP" if packet.haslayer(ICMP) else "Other",
            "packet_size": len(packet)
        }
        packet_list.append(packet_data)
        print(f"Captured Packet: {packet_data}")  # Print captured packet info

# Capture a defined number of packets for testing (adjust count as needed)
sniff(filter="ip", prn=packet_handler, count=500)

# Convert captured packets to DataFrame
df = pd.DataFrame(packet_list)

# Save to CSV for analysis
df.to_csv("network_traffic.csv", index=False)
print("✅ Packet capture complete! Data saved to network_traffic.csv")


Captured Packet: {'timestamp': 1739266082.331805, 'src_ip': '192.168.2.12', 'dst_ip': '192.168.2.1', 'src_port': 64659, 'dst_port': 53, 'protocol': 'UDP', 'packet_size': 92}
Captured Packet: {'timestamp': 1739266082.3388329, 'src_ip': '192.168.2.1', 'dst_ip': '192.168.2.12', 'src_port': 53, 'dst_port': 64659, 'protocol': 'UDP', 'packet_size': 215}
Captured Packet: {'timestamp': 1739266082.3399749, 'src_ip': '192.168.2.12', 'dst_ip': '20.42.73.24', 'src_port': 50355, 'dst_port': 443, 'protocol': 'TCP', 'packet_size': 78}
Captured Packet: {'timestamp': 1739266082.3688269, 'src_ip': '20.42.73.24', 'dst_ip': '192.168.2.12', 'src_port': 443, 'dst_port': 50355, 'protocol': 'TCP', 'packet_size': 74}
Captured Packet: {'timestamp': 1739266082.3690639, 'src_ip': '192.168.2.12', 'dst_ip': '20.42.73.24', 'src_port': 50355, 'dst_port': 443, 'protocol': 'TCP', 'packet_size': 66}
Captured Packet: {'timestamp': 1739266082.3693209, 'src_ip': '192.168.2.12', 'dst_ip': '20.42.73.24', 'src_port': 50355, '

In [8]:
import pandas as pd

# Load captured packet data
df = pd.read_csv("network_traffic.csv")

# Count occurrences of each source and destination IP
df["src_ip_count"] = df.groupby("src_ip")["src_ip"].transform("count")
df["dst_ip_count"] = df.groupby("dst_ip")["dst_ip"].transform("count")

# Calculate time differences between packets (useful for detecting bursts)
df["time_between_packets"] = df["timestamp"].diff().fillna(0)

# Drop unnecessary columns
df = df.drop(columns=["src_ip", "dst_ip", "timestamp"])

# Save processed data
df.to_csv("processed_traffic.csv", index=False)

print("✅ Feature engineering complete! Data saved to processed_traffic.csv")


✅ Feature engineering complete! Data saved to processed_traffic.csv


In [4]:
from sklearn.ensemble import IsolationForest
import joblib

# Load processed network traffic data
df = pd.read_csv("processed_traffic.csv")

# Select relevant features
features = ["packet_size", "src_ip_count", "dst_ip_count", "is_tcp", "is_udp", "time_between_packets"]
X = df[features]

# Train Isolation Forest Model
model = IsolationForest(n_estimators=100, contamination=0.02, random_state=42)
model.fit(X)

# Save model
joblib.dump(model, "isolation_forest_model.pkl")

print("✅ Model trained and saved as isolation_forest_model.pkl")


✅ Model trained and saved as isolation_forest_model.pkl


In [6]:
from scapy.all import sniff, IP, TCP, UDP
import joblib
import pandas as pd

# Load trained Isolation Forest model
model = joblib.load("isolation_forest_model.pkl")

# Define the expected feature order
expected_features = ["packet_size", "src_ip_count", "dst_ip_count", "is_tcp", "is_udp", "time_between_packets"]

packet_history = []  # Store previous packet timestamps for time-based features

def detect_anomaly(packet):
    if packet.haslayer(IP):  # Process only IP packets
        packet_data = {
            "packet_size": len(packet),
            "src_ip_count": 1,  # Assume real-time analysis
            "dst_ip_count": 1,
            "is_tcp": 1 if packet.haslayer(TCP) else 0,
            "is_udp": 1 if packet.haslayer(UDP) else 0,
            "time_between_packets": 0 if len(packet_history) == 0 else packet.time - packet_history[-1]
        }

        # Keep track of time for time-based feature
        packet_history.append(packet.time)

        # Convert to DataFrame and **ensure the correct feature order**
        df_packet = pd.DataFrame([packet_data])[expected_features]

        # Predict if this packet is an anomaly
        prediction = model.predict(df_packet)

        if prediction[0] == -1:  # Anomaly detected
            print("🚨 Anomaly Detected! 🚨")
            print(packet.summary())

# Start live packet capture and anomaly detection
sniff(filter="ip", prn=detect_anomaly)


🚨 Anomaly Detected! 🚨
Ether / IP / TCP 142.251.40.138:https > 192.168.2.12:49206 PA / Raw
🚨 Anomaly Detected! 🚨
Ether / IP / UDP 192.168.2.1:9431 > 192.168.2.255:9431 / Raw
🚨 Anomaly Detected! 🚨
Ether / IP / UDP 192.168.2.12:56822 > 142.250.72.110:https / Raw
🚨 Anomaly Detected! 🚨
Ether / IP / UDP 192.168.2.12:56822 > 142.250.72.110:https / Raw
🚨 Anomaly Detected! 🚨
Ether / IP / UDP 192.168.2.12:56822 > 142.250.72.110:https / Raw
🚨 Anomaly Detected! 🚨
Ether / IP / UDP 192.168.2.12:56822 > 142.250.72.110:https / Raw
🚨 Anomaly Detected! 🚨
Ether / IP / UDP 192.168.2.12:56822 > 142.250.72.110:https / Raw
🚨 Anomaly Detected! 🚨
Ether / IP / UDP 192.168.2.12:56822 > 142.250.72.110:https / Raw
🚨 Anomaly Detected! 🚨
Ether / IP / UDP 192.168.2.12:56822 > 142.250.72.110:https / Raw
🚨 Anomaly Detected! 🚨
Ether / IP / UDP 142.250.72.110:https > 192.168.2.12:56822 / Raw
🚨 Anomaly Detected! 🚨
Ether / IP / UDP 142.250.72.110:https > 192.168.2.12:56822 / Raw
🚨 Anomaly Detected! 🚨
Ether / IP / UDP 142.2

<Sniffed: TCP:384 UDP:107 ICMP:0 Other:0>